In [1]:
import os
from path import Path
import pickle
import numpy as np
import pandas as pd
import spacy

In [2]:
!python -m spacy download en_core_web_md

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


2021-03-10 03:27:30.461491: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-03-10 03:27:30.461615: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
data = 'resources/mbti_clean.csv'
df = pd.read_csv(data)
df.head()

,Unnamed: 0,type,posts
0,6,INFJ,The last thing INFJ friend posted his facebo...
1,7,INFJ,Hello ENFJ Sorry hear your distress Its only...
2,11,INFJ,Prozac wellbrutin least thirty minutes movin...
3,12,INFJ,Basically come with three items youve determi...
4,13,INFJ,All things moderation Sims indeed video ga...


In [4]:
embedding_dim = 300
X = np.empty((df.shape[0], embedding_dim))

In [5]:
def doc2vec(document: str):
    nlp = spacy.load("en_core_web_md")
    doc = nlp(document)
    return doc.vector

In [ ]:
for i, doc in enumerate(df["posts"]):
    if i % 1000 == 0:
        print(f"Completed {i} conversions")
    X[i] = doc2vec(doc)

In [6]:
import pickle

with open("embeddings.pkl", "wb") as f:
    pickle.dump(X, f)

In [7]:
if os.path.exists("embeddings.pkl"):
    with open("embeddings.pkl", "rb") as f:
        X = pickle.load(f)
else:
    for i, doc in enumerate(df["posts"]):
        if i % 1000 == 0:
            print(f"Completed {i} conversions")
        X[i] = doc2vec(doc)

In [8]:
y = df["type"]

In [9]:
X = X[0 : 10000] 
y = y[0 : 10000]

In [10]:
# Split the dataset into training and testing sets

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
   y,  random_state=1, stratify=y)
X_train.shape

(7500, 300)

In [11]:
from sklearn.svm import SVC
model = SVC(kernel='linear')

In [12]:
# Train the model
model.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [13]:
# Create predictions with the model
y_pred = model.predict(X_test)
results = pd.DataFrame({
   "Prediction": y_pred,
   "Actual": y_test
}).reset_index(drop=True)
results.head()

,Prediction,Actual
0,INFP,ISTJ
1,INFP,INFJ
2,INFP,INTP
3,INFP,INFP
4,INFP,INTP


In [14]:
# Assess the accuracy_score of the model
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.2276

In [15]:
# Generate confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

        ENFJ       0.00      0.00      0.00        74
        ENFP       0.00      0.00      0.00       174
        ENTJ       0.00      0.00      0.00        87
        ENTP       0.00      0.00      0.00       158
        ESFP       0.00      0.00      0.00         5
        ESTP       0.00      0.00      0.00        12
        INFJ       0.00      0.00      0.00       404
        INFP       0.23      1.00      0.37       569
        INTJ       0.00      0.00      0.00       297
        INTP       0.00      0.00      0.00       482
        ISFJ       0.00      0.00      0.00        37
        ISFP       0.00      0.00      0.00        74
        ISTJ       0.00      0.00      0.00        53
        ISTP       0.00      0.00      0.00        74

    accuracy                           0.23      2500
   macro avg       0.02      0.07      0.03      2500
weighted avg       0.05      0.23      0.08      2500



C:\Users\quija\anaconda3\envs\PythonData\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
